In [2]:
import pandas
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from collections import Counter
import time
import seaborn as sns
from matplotlib import pyplot as plt

import tensorflow as tf
#from keras.utils import to_categorical
import feather
import random
from datetime import *

import warnings
warnings.filterwarnings("ignore")

In [48]:
FILENAME = "Datasets/Round 3/preproccesed_outliers_131362_col_7_threshold_0.8_"
data = feather.read_dataframe("{}.feather".format(FILENAME))
data = data.dropna().reset_index(drop=True)

In [19]:
import censusdata

county = censusdata.download('acs5', 2018, censusdata.censusgeo([('county', '*')]),
                                   ['DP05_0001E',  'DP03_0019PE','DP03_0021PE','DP03_0022PE','DP03_0062E'],
                                   tabletype='profile')

In [20]:
# rename columns
county.columns = ['Population_County','Drive_County','Transit_County','Walk_County','MedianHouseholdIncome_County']
county = county.reset_index()
# extract county name and convert them to lowercase
county['County_y'] = county['index'].apply(lambda x : x.name.split(' County')[0].split(',')[0]).str.lower()
county = county.drop('index',axis=1)
county

,Population_County,Drive_County,Transit_County,Walk_County,MedianHouseholdIncome_County,County_y
0,47086,86.4,0.0,1.3,30834,washington
1,12028,85.8,0.0,1.8,39007,perry
2,8321,85.6,0.3,1.1,37203,choctaw
3,23480,82.4,0.2,0.7,40510,itawamba
4,10129,90.0,0.0,1.4,43060,carroll
5,21278,85.7,0.3,1.8,36877,grenada
6,7346,92.4,0.0,1.9,20188,jefferson
7,13714,82.5,0.0,0.6,49447,greene
8,35787,86.4,0.5,1.4,41572,marshall
9,7372,84.0,0.0,2.6,25510,quitman


In [50]:
# Convert all county values in data to lowercase

data['County'] = data['County'].str.lower()

In [ ]:
counties = []
for i in data['County'].values:
    counties.append(i.replace(' county', ''))
data['County'] = counties

### This is the code that destroys my RAM

In [ ]:
sample_data = 0

# left join df with census data
data = data.merge(county, left_on = 'County', right_on='County_y').drop('County_y', axis=1)
data

In [51]:
# counties = []
# for i in data['County'].values:
#     counties.append(i.replace(' county', ''))

# count = 0
# not_in = []
# for i in set(counties):
#     if i in county['County_y'].values:
#         count+=1
#     else:
#         not_in.append(i)
# print(count)

1584
